# ベイズ線形回帰（MCMC）

教師あり学習では{ (x_i,y_i) }のようなラベルの付いたデータからデータの隠された構造を推定するのが目的です。表題の回帰という語は出力変数yが連続的なときに用いられることが多いです。

ここでは[ベイズ線形回帰（教師あり学習)](BayesianLinearRegression.ipynb)と同様の解析をマルコフ連鎖モンテカルロ法(MCMC)を用いて行います。

変分ベイズがモデルをを近似した関数を　MCMCはサンプリングによって事後分布を推定します。

In [7]:
import tensorflow as tf
import edward as ed

# Data

変分ベイズの場合と同じ訓練データ、テストデータを用意します。

In [21]:
import numpy as np

def build_toy_dataset(N, w, noise_std=0.1):
  D = len(w)
  x = np.random.randn(N, D)
  y = np.dot(x, w) + np.random.normal(0, noise_std, size=N)
  return x, y

N = 40  # number of data points
D = 10  # number of features

w_true = np.random.randn(D) #係数wの真の値
X_train, y_train = build_toy_dataset(N, w_true)
X_test, y_test = build_toy_dataset(N, w_true)

In [24]:
len(y_train)


40

# Model

モデルも以前と同じでここではベイズ線形回帰モデル(Murphy, 2012[^1])を仮定します。入力$x_n \in \mathbb{R}^{D}$, 出力$y_n \in \mathbb{R}$間には線形の関係があるとします。

N個のデータ点
$(\mathbf{X},\mathbf{y})=\{(\mathbf{x}_n, y_n)\}(X,y)={(x_n,y_n}) $に対しモデルは以下の分布関数で表される関係を仮定しています。
\begin{aligned} p(\mathbf{w}) &= \text{Normal}(\mathbf{w} \mid \mathbf{0}, \sigma_w^2\mathbf{I}), \\[1.5ex] p(b) &= \text{Normal}(b \mid 0, \sigma_b^2), \\ p(\mathbf{y} \mid \mathbf{w}, b, \mathbf{X}) &= \prod_{n=1}^N \text{Normal}(y_n \mid \mathbf{x}_n^\top\mathbf{w} + b, \sigma_y^2).\end{aligned}

この線形モデルの隠れた変数は重み$\mathbf{w}$と切片(バイアスとも言う)b です。$\sigma_w^2,\sigma_b^2$は事前分布の分散、$\sigma_y^2$
は尤度の分散です。尤度の平均は入力 $\mathbf{x}_nx$ の線形変換として与えられています。

Edwardでモデルを書きましょう。$\sigma_w,\sigma_b,\sigma_y=1$
と固定すると

In [30]:
from edward.models import Normal

X = tf.placeholder(tf.float32, [N, D])
w = Normal(loc=tf.zeros(D), scale=tf.ones(D))
b = Normal(loc=tf.zeros([]), scale=tf.ones([]))
y = Normal(loc=ed.dot(X, w) + b, scale=tf.ones(N))

と書けます。ここでXはtensorflowのplaceholderm[^2]です。推論処理の間、データに応じた値をこのpalceholderに渡すことになります。

# 推論

推論ではEmpiricalクラスを用いて推定対象のモデルの変数を定義します。

In [31]:
from edward.models import Empirical
T = 5000  # number of samples
qw = Empirical(params=tf.Variable(tf.random_normal([T, D])))
qb = Empirical(params=tf.Variable(tf.random_normal([T])))

ここでは変分推論と時と同じ因子化された変数をqwとqbを使っています(これがベストなのかは判断が分かれます)。

## HMC

In [32]:
inference_HMC = ed.HMC({w: qw, b: qb}, data={X: X_train, y: y_train})
inference_HMC.initialize(n_print=10, step_size=0.6)

wとqwの次元があっていないと以下のようなエラーが出てしまうので合わせたほうが良いです。

https://discourse.edwardlib.org/t/bayesian-hacker-chapter1-introduction-imp-use-edward/299

In [57]:
inference_HMC.run(step_size=1e-3)
qw_HMC,qb_HMC=qw,qb
y_post_SGHMC = ed.copy(y, {w: qw, b: qb})
#y_test=[int(i) for i in y_test]

5000/5000 [100%] ██████████████████████████████ Elapsed: 5s | Acceptance Rate: 1.000


HMCの他にMetropolisHastings法やGibbsサンプリングも使うことが出来ます。所要時間とAcceptance rateを見るとHMCが相対的に収束が早いことがわかる。

In [59]:
inference_MH = ed.MetropolisHastings({w: qw, b: qb},{w:w,b:b},data={X: X_train, y: y_train})
#inference_MH.initialize(n_print=10, step_size=0.6)
inference_MH.run()
qw_MH,qb_MH=qw,qb
y_post_MH = ed.copy(y, {w: qw, b: qb})

5000/5000 [100%] ██████████████████████████████ Elapsed: 5s | Acceptance Rate: 0.002


In [60]:
inference_Gibbs = ed.Gibbs({w: qw, b: qb},{w:w,b:b}, data={X: X_train, y: y_train})
#inference_Gibbs.initialize(n_print=10, step_size=0.6)
inference_Gibbs.run()
qw_Gibbs,qb_Gibbs=qw,qb
y_post_Gibbs = ed.copy(y, {w: qw, b: qb})

5000/5000 [100%] ██████████████████████████████ Elapsed: 6s | Acceptance Rate: 1.000


# Criticism

In [61]:
def printMSE(X,X_test,y_post,y_test):
    print("Mean squared error on test data:")
    print(ed.evaluate('mean_squared_error', data={X: X_test, y_post: y_test}))

In [64]:
printMSE(X,X_test,y_post_SGHMC,y_test)

Mean squared error on test data:
12.4283


In [66]:
printMSE(X,X_test,y_post_MH,y_test)

Mean squared error on test data:
12.2556


In [65]:
printMSE(X,X_test,y_post_Gibbs,y_test)

Mean squared error on test data:
12.3137
